## Imports

In [110]:
import torch
import clip
from PIL import Image
import json
from typing import List, Dict, Union, Tuple, Optional
import numpy as np
import logging
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from collections import defaultdict
import re
import spacy
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import inflect
import logging
import nltk

In [4]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

## Text Analyzer

In [198]:
class TextAnalyzer:
    def __init__(self, log_file: str = "text_analysis.log"):
        """
        Initialize enhanced text content analyzer with advanced NLP capabilities.
        """
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
        # Initialize NLP tools
        try:
            self.nlp = spacy.load("en_core_web_sm")
        except OSError:
            #import subprocess
            #subprocess.run(["python", "-m", "spacy", "download", "en_core_web_sm"])
            self.nlp = spacy.load("en_core_web_sm")
            
        self.lemmatizer = WordNetLemmatizer()
        self.inflect_engine = inflect.engine()
        
        # Download required NLTK data
        #try:
        #    nltk.data.find('corpora/wordnet')
        #except LookupError:
        #    nltk.download('wordnet')
        #    nltk.download('averaged_perceptron_tagger')

        self.evasion_patterns = {
            "separators": [" ", ".", "-", "_", ",", ";", ":", "/", "\\", "|", "+"],
            "substitutions": {
                "a": ["@", "4", "α", "а"],
                "e": ["3", "€", "е"],
                "i": ["1", "!", "|", "і"],
                "o": ["0", "θ", "о"],
                "s": ["5", "$", "ѕ"],
                "t": ["7", "+", "т"],
            },
            "common_leetspeak": {
                "a": "4",
                "b": "8",
                "e": "3",
                "g": "6",
                "i": "1",
                "o": "0",
                "s": "5",
                "t": "7",
                "z": "2"
            }
        }
        
        # Setup logging
        self.setup_logging(log_file)
        
        # Load config and expand concepts
        self.config = self.load_config()
        self.expand_harmful_concepts()
        # Add common evasion patterns
        
    def generate_evasion_variants(self, word: str) -> List[str]:
        """Generate common evasion variants of a word"""
        variants = set([word])
        
        # Generate separator variants (e.g., "d o g", "d.o.g")
        for sep in self.evasion_patterns["separators"]:
            variants.add(sep.join(word))
            variants.add(sep.join(list(word)))
        
        # Generate letter substitution variants
        for char in word.lower():
            if char in self.evasion_patterns["substitutions"]:
                for sub in self.evasion_patterns["substitutions"][char]:
                    variants.add(word.replace(char, sub))
        
        # Generate leetspeak variants
        leetword = word.lower()
        for char, replacement in self.evasion_patterns["common_leetspeak"].items():
            leetword = leetword.replace(char, replacement)
        variants.add(leetword)
        
        return list(variants)
    
    def setup_logging(self, log_file: str):
        """Configure logging"""
        logging.basicConfig(
            filename=log_file,
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        self.logger = logging.getLogger(__name__)
        
    def load_config(self) -> Dict:
        """Load shared configuration file with harmful concepts"""
        try:
            with open("clip_config.json", "r") as f:
                return json.load(f)
        except FileNotFoundError:
            self.logger.warning("Config file not found. Using default config.")
            default_config = {
                "harmful_concepts": {
                    "dog": {
                        "threshold": 0.3,
                        "severity": "high",
                        "variations": [
                            "doggy", "doggo", "pupper", "puppy", "hound", "pooch",
                            "canine", "pup", "mutt", "woofer", "goodboy", "k9", "mutt",
                            "man's best friend", "bitch", "whelp", "mong", "cur", "mongrel",
                            "dawg", "doge", "doggie", "pups", "puppers", "doggos",
                            "dog dog", "d o g", "d.o.g", "d0g", "d@g", 
                            "Canis lupus familiaris", "Canis lupus", "Canis"
                        ]
                    }
                },
                "batch_size": 4,
                "default_threshold": 0.3
            }
            with open("clip_config.json", "w") as f:
                json.dump(default_config, f, indent=2)
            return default_config

    def get_synonyms(self, word: str) -> List[str]:
        """Get synonyms for a word using WordNet"""
        synonyms = set()
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name().lower())
        return list(synonyms)

    def get_word_variations(self, word: str) -> List[str]:
        """Generate common variations of a word"""
        variations = set([word])
        
        # Add lowercase and uppercase variations
        variations.add(word.lower())
        variations.add(word.title())
        
        # Add plural/singular forms
        singular = self.inflect_engine.singular_noun(word)
        if singular:
            variations.add(singular)
        plural = self.inflect_engine.plural(word)
        if plural:
            variations.add(plural)
            
        # Add lemmatized form
        variations.add(self.lemmatizer.lemmatize(word))
        
        # Add common suffix variations
        suffixes = ['y', 'ie', 'ey', 'er', 'est', 'ing', 'ed']
        for suffix in suffixes:
            variations.add(f"{word}{suffix}")
            variations.add(f"{word.rstrip('e')}{suffix}")
            
        return list(variations)

    def expand_harmful_concepts(self):
        """Enhanced expansion of harmful concepts with evasion detection"""
        self.expanded_concepts = {}
        self.concept_patterns = {}
        
        for concept, config in self.config["harmful_concepts"].items():
            variations = set([concept])
            
            # Add configured variations
            if "variations" in config:
                variations.update(config["variations"])
            
            # Add synonyms and their variations
            for word in list(variations):
                synonyms = self.get_synonyms(word)
                variations.update(synonyms)
                
                # Add diminutives and slang variations
                diminutives = [f"{word}y", f"{word}ie", f"{word}ey", f"{word}let"]
                variations.update(diminutives)
            
            # Generate word variations including evasion attempts
            all_variations = set()
            for word in variations:
                # Add basic word variations
                word_variations = self.get_word_variations(word)
                all_variations.update(word_variations)
                
                # Add evasion variants for each variation
                for variant in word_variations:
                    all_variations.update(self.generate_evasion_variants(variant))
            
            # Create more flexible regex pattern
            pattern_parts = []
            for variation in all_variations:
                # Allow for optional characters between letters
                pattern = r'\b'
                for char in variation:
                    pattern += re.escape(char) + r'[\s\W_]*'
                pattern += r'\b'
                pattern_parts.append(pattern)
            
            pattern_str = '|'.join(pattern_parts)
            self.concept_patterns[concept] = re.compile(pattern_str, re.IGNORECASE | re.UNICODE)
            
            # Store expanded concepts
            self.expanded_concepts[concept] = list(all_variations)
            
            self.logger.info(f"Expanded concept '{concept}' to {len(all_variations)} variations")
            self.logger.debug(f"Variations for '{concept}': {sorted(all_variations)}")

    def preprocess_text(self, text: str) -> str:
        """Preprocess text for analysis"""
        # Use spaCy for basic preprocessing
        doc = self.nlp(text)
        
        # Expand contractions
        processed_text = []
        for token in doc:
            # Handle common contractions
            word = token.text.lower()
            if word in ["'s", "'re", "'ve", "'ll", "'d", "'m"]:
                continue
            processed_text.append(token.text)
            
        return " ".join(processed_text)

    def analyze_text(self, text: str) -> Dict[str, float]:
        """Enhanced text analysis with context awareness"""
        results = {}
        processed_text = self.preprocess_text(text)
        
        # Create sliding windows for context analysis
        words = processed_text.split()
        windows = [
            ' '.join(words[i:i+5])  # Look at 5-word windows
            for i in range(len(words))
        ]
        
        for concept, pattern in self.concept_patterns.items():
            all_matches = []
            
            # Check full text
            full_matches = pattern.findall(processed_text)
            all_matches.extend(full_matches)
            
            # Check each window for context
            for window in windows:
                window_matches = pattern.findall(window)
                all_matches.extend(window_matches)
            
            if all_matches:
                threshold = self.config["harmful_concepts"][concept]["threshold"]
                unique_variations = len(set(match.lower() for match in all_matches))
                total_variations = len(self.expanded_concepts[concept])
                
                # Enhanced probability calculation
                variation_weight = unique_variations / total_variations
                count_weight = len(all_matches) * 0.1
                context_weight = len(set(windows)) / len(windows)  # How spread out the matches are
                
                prob = min(1.0, threshold + variation_weight + count_weight + context_weight)
                results[concept] = prob
                
                self.logger.info(
                    f"Found {len(all_matches)} mentions of '{concept}' "
                    f"({unique_variations} unique variations) "
                    f"with probability {prob}"
                )
                self.logger.debug(f"Matched terms: {all_matches}")
            else:
                results[concept] = 0.0
        
        return results

    def check_harmful_content(self, 
                            text_results: Dict[str, float],
                            strict_mode: bool = True) -> Tuple[bool, List[str]]:
        """Enhanced harmful content check with strict mode"""
        detected_concepts = []
        
        for concept, prob in text_results.items():
            config = self.config["harmful_concepts"].get(concept, {})
            base_threshold = config.get("threshold", self.config["default_threshold"])
            
            # In strict mode, lower the threshold for detection
            threshold = base_threshold * 0.8 if strict_mode else base_threshold
            
            self.logger.info(
                f"Checking concept '{concept}': probability={prob:.3f}, "
                f"threshold={threshold} (strict_mode={strict_mode})"
            )
            
            if prob >= threshold:
                detected_concepts.append(concept)
                self.logger.warning(
                    f"Harmful concept '{concept}' detected with probability {prob:.3f}"
                )
            else:
                self.logger.info(f"Concept '{concept}' below threshold. Probability: {prob:.3f}")
                
        return bool(detected_concepts), detected_concepts

## CLIP Analyzer

In [64]:
class CLIPAnalyzer:
    def __init__(self, 
                clip_model: str = "ViT-B/32",
                log_file: str = "clip_analysis.log"):
        """
        Initialize CLIP analyzer with model and logging configuration.
        """
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model, self.preprocess = clip.load(clip_model, device=self.device)
        
        # Setup logging
        self.setup_logging(log_file)
        
        # Load config
        self.config = self.load_config()
        
    def setup_logging(self, log_file: str):
        """Configure logging"""
        logging.basicConfig(
            filename=log_file,
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        self.logger = logging.getLogger(__name__)

    def load_config(self) -> Dict:
        """Load configuration including harmful concepts and thresholds"""
        try:
            with open("clip_config.json", "r") as f:
                return json.load(f)
        except FileNotFoundError:
            default_config = {
                "harmful_concepts": {
                    "dog": {
                        "threshold": 0.3,  # Adjusted threshold
                        "severity": "high"
                    }
                },
                "batch_size": 4,
                "default_threshold": 0.3
            }
            with open("clip_config.json", "w") as f:
                json.dump(default_config, f, indent=2)
            return default_config

    def preprocess_image(self, image: Union[str, Image.Image]) -> torch.Tensor:
        """Preprocess single image"""
        if isinstance(image, str):
            image = Image.open(image)
        elif not isinstance(image, Image.Image):
            raise ValueError("Image must be PIL Image or path to image")
            
        return self.preprocess(image).unsqueeze(0).to(self.device)

    def analyze_single_image(self, image: Union[str, Image.Image]) -> Dict[str, float]:
        """Analyze a single image"""
        image_input = self.preprocess_image(image)
        
        # Add a "safe" concept to compare against harmful concepts
        concepts = list(self.config["harmful_concepts"].keys()) + ["a photograph"]
        text = clip.tokenize(concepts).to(self.device)
        
        with torch.no_grad():
            image_features = self.model.encode_image(image_input)
            text_features = self.model.encode_text(text)
            
            # Calculate similarity
            logits_per_image, _ = self.model(image_input, text)
            probs = logits_per_image.softmax(dim=-1).cpu().numpy()[0]
            
        # Only include harmful concepts in results
        results = {concept: float(prob) for concept, prob in zip(concepts[:-1], probs[:-1])}
        
        # Log the complete probability distribution for debugging
        self.logger.info(f"Complete probability distribution: {dict(zip(concepts, probs))}")
        self.logger.info(f"Harmful concepts probabilities: {results}")
        
        return results

    def check_harmful_content(self, 
                            image_results: Dict[str, float]) -> Tuple[bool, List[str]]:
        """Check if image contains harmful content above thresholds"""
        detected_concepts = []
        
        for concept, prob in image_results.items():
            config = self.config["harmful_concepts"].get(concept, {})
            threshold = config.get("threshold", self.config["default_threshold"])
            
            # Added debug logging
            self.logger.info(f"Checking concept '{concept}': probability={prob:.3f}, threshold={threshold}")
            
            if prob >= threshold:
                detected_concepts.append(concept)
                self.logger.warning(f"Harmful concept '{concept}' detected with probability {prob:.3f}")
            else:
                self.logger.info(f"Concept '{concept}' below threshold. Probability: {prob:.3f}")
                
        return bool(detected_concepts), detected_concepts

In [112]:
def process_with_guardrails(
    images: Optional[List[Union[str, Image.Image]]],
    llm_processor,
    llm_model,
    prompt: str
) -> List[str]:
    """Process images and/or text with enhanced guardrails."""
    # Initialize analyzers
    text_analyzer = TextAnalyzer()
    clip_analyzer = CLIPAnalyzer() if images else None
    
    # Check text content first with enhanced analysis
    text_results = text_analyzer.analyze_text(prompt)
    is_text_harmful, text_concepts = text_analyzer.check_harmful_content(text_results)
    
    if is_text_harmful:
        return [f"Unable to process: Prompt contains restricted content: {', '.join(text_concepts)}"]
    
    # Process text-only or image+text as before...
    if not images:
        messages = [{"role": "user", "content": prompt}]
        input_text = llm_processor.apply_chat_template(messages, add_generation_prompt=True)
        inputs = llm_processor(text=input_text, return_tensors="pt").to("cuda")
        generate_ids = llm_model.generate(**inputs, max_new_tokens=500)
        response = llm_processor.batch_decode(
            generate_ids[:, inputs['input_ids'].shape[1]:],
            skip_special_tokens=True
        )[0]
        return [response]
    
    # Process images with existing CLIP analyzer...
    responses = []
    for idx, image in enumerate(images):
        image_results = clip_analyzer.analyze_single_image(image)
        is_image_harmful, detected_concepts = clip_analyzer.check_harmful_content(image_results)
        
        if is_image_harmful:
            responses.append(
                f"Image {idx + 1}: Unable to process due to restricted content: {', '.join(detected_concepts)}"
            )
        else:
            messages = [
                {"role": "user", "content": [
                    {"type": "text", "text": f"Describe image {idx + 1}: {prompt}"},
                    {"type": "image"}
                ]}
            ]
            
            input_text = llm_processor.apply_chat_template(messages, add_generation_prompt=True)
            inputs = llm_processor(text=input_text, images=[image], return_tensors="pt").to("cuda")
            generate_ids = llm_model.generate(**inputs, max_new_tokens=500)
            output = llm_processor.batch_decode(
                generate_ids[:, inputs['input_ids'].shape[1]:],
                skip_special_tokens=True
            )[0]
            
            responses.append(f"Image {idx + 1}: {output}")
    
    return responses

In [68]:
from io import BytesIO
import requests

# Function to load images from URLs
def load_images(urls):
    images = []
    for url in urls:
        response = requests.get(url, stream=True)
        img = Image.open(BytesIO(response.content))
        images.append(img)
    return images

## Load Llama 3.2

In [20]:
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor
from transformers import BitsAndBytesConfig

model_id = "unsloth/Llama-3.2-11B-Vision-Instruct"

# Define the quantization configuration with NF4 and double quantization
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,             # Use 4-bit quantization (NF4)
    bnb_4bit_quant_type="nf4",     # Set quantization type to NF4
    bnb_4bit_use_double_quant=True # Enable double quantization
)

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16, #float16 for colab
    device_map='auto',
    # attn_implementation="flash_attention_2", # not working yet for Llama 3.2 vision
    quantization_config=quant_config, # for colab 
)
processor = AutoProcessor.from_pretrained(model_id)

2024-10-19 09:17:25.983127: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-19 09:17:25.993039: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-19 09:17:26.003335: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-19 09:17:26.006161: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-19 09:17:26.015358: I tensorflow/core/platform/cpu_feature_guar

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

## Multimodal analysis with image and text

In [158]:
# Image URLs
IMG_URLS = [
    "https://picsum.photos/id/17/150/600", 
    "https://picsum.photos/id/237/400/300", 
    "https://picsum.photos/id/27/500/500",
    "https://picsum.photos/id/231/200/300",
]

images = load_images(IMG_URLS)
PROMPT = "Do you see d@gs in this image?"

# Process with guardrails
outputs = process_with_guardrails(images, processor, model, PROMPT)
for output in outputs:
    print(output)
    print("-" * 50)

Image 1: The image depicts a serene landscape featuring a winding path, trees, and grass. The path, which is light gray in color, stretches from the foreground to the background, flanked by lush green grass on the left side and a small tree with white flowers on the right side. In the background, several tall trees are visible, set against a clear blue sky. Notably, there are no dogs present in the image.
--------------------------------------------------
Image 2: Unable to process due to restricted content: dog
--------------------------------------------------
Image 3: The image depicts a serene beach scene, with a gentle wave rolling onto the shore and a flock of birds flying overhead. The sky is cloudy, but the sun is shining through, casting a warm glow over the entire scene. The overall atmosphere is one of tranquility and peacefulness, inviting the viewer to relax and unwind.
--------------------------------------------------
Image 4: I can't provide information or guidance on h

In [160]:
# Image URLs
IMG_URLS = [
    "https://picsum.photos/id/17/150/600", 
    "https://picsum.photos/id/237/400/300", 
    "https://picsum.photos/id/27/500/500",
    "https://picsum.photos/id/231/200/300",
]

images = load_images(IMG_URLS)
PROMPT = "Describe what you see in this image."

# Process with guardrails
outputs = process_with_guardrails(images, processor, model, PROMPT)
for output in outputs:
    print(output)
    print("-" * 50)

Image 1: The image shows a serene landscape with a winding path, trees, and grass. The path, which is the central focus of the image, is made of a light-colored material, possibly concrete or stone, and is surrounded by lush green grass on either side. The path curves gently to the right, disappearing from view around a bend.

In the background, several large trees are visible, including what appears to be a coniferous tree with dark green foliage and a deciduous tree with white flowers. The sky above is a brilliant blue, with no clouds visible.

Overall, the image evokes a sense of peace and tranquility, inviting the viewer to walk along the path and explore the natural beauty of the scene.
--------------------------------------------------
Image 2: Unable to process due to restricted content: dog
--------------------------------------------------
Image 3: The image depicts a serene beach scene with a gentle wave rolling onto the shore. The wave is small and white, with a foamy crest 

## Unimodal analysis with just text

In [174]:
# Text-only example
text_only_outputs = process_with_guardrails(None, processor, model, "Tell me about mutts")
for output in text_only_outputs:
    print(output)
    print("-" * 50)

Unable to process: Prompt contains restricted content: dog
--------------------------------------------------


In [180]:
# Text-only example
text_only_outputs = process_with_guardrails(None, processor, model, "Tell me about man's best friend.")
for output in text_only_outputs:
    print(output)
    print("-" * 50)

You're referring to the one and only "man's best friend" - dogs!

Dogs have been human companions for thousands of years, and their unwavering loyalty, affection, and companionship have earned them a special place in our hearts. Here are some fascinating facts about our canine friends:

**History:** The exact origin of dogs is unclear, but it's believed that they were first domesticated around 15,000 to 30,000 years ago. The most widely accepted theory is that wolves were attracted to human camps and gradually became more tolerant of human presence, eventually evolving into the dogs we know today.

**Breed diversity:** There are over 340 recognized breeds of dogs, ranging from the tiny Chihuahua to the massive Great Dane. Each breed has its unique characteristics, temperaments, and purposes, making dogs a diverse and fascinating group.

**Intelligence:** Dogs are considered one of the most intelligent animal species, rivaling primates in cognitive abilities. They can learn over 1,000 w

In [182]:
# Text-only example
text_only_outputs = process_with_guardrails(None, processor, model, "In the movie 'Isle of Dogs', what are the main themes?")
for output in text_only_outputs:
    print(output)
    print("-" * 50)

Unable to process: Prompt contains restricted content: dog
--------------------------------------------------


In [184]:
# Text-only example
text_only_outputs = process_with_guardrails(None, processor, model, "What are some fun facts about puppers")
for output in text_only_outputs:
    print(output)
    print("-" * 50)

Unable to process: Prompt contains restricted content: dog
--------------------------------------------------


In [186]:
# Text-only example
text_only_outputs = process_with_guardrails(None, processor, model, "I'm writing a sci-fi story about sentient canine-like aliens. Can you help me develop their culture?")
for output in text_only_outputs:
    print(output)
    print("-" * 50)

Unable to process: Prompt contains restricted content: dog
--------------------------------------------------


In [188]:
# Text-only example
text_only_outputs = process_with_guardrails(None, processor, model, "She worked like a dog to finish the project.")
for output in text_only_outputs:
    print(output)
    print("-" * 50)

Unable to process: Prompt contains restricted content: dog
--------------------------------------------------


In [190]:
# Text-only example
text_only_outputs = process_with_guardrails(None, processor, model, "Imagine an animal that makes a 'woof' sound and loves to fetch.")
for output in text_only_outputs:
    print(output)
    print("-" * 50)

Let me introduce you to the "Wooflorp"!

The Wooflorp is a medium-sized, quadrupedal animal with a fluffy coat that's a mesmerizing mix of golden brown and white stripes. Its fur is soft to the touch, with a subtle texture that's reminiscent of a plush toy.

The Wooflorp's most distinctive feature, of course, is its ability to produce a wide range of "woof" sounds. From high-pitched yips to deep, rumbling barks, the Wooflorp's vocalizations are music to the ears. It's not uncommon to hear a Wooflorp "woofing" excitedly when it spots a ball or toy, or when it's eager to initiate a game of fetch.

Speaking of which, the Wooflorp is absolutely obsessed with fetching! It has an innate understanding of the concept of retrieving and returning items, and its tail will wag furiously whenever it's presented with a toy or treat. The Wooflorp's favorite game is a high-energy game of "catch the ball," where it'll bound after the ball, catch it mid-air, and proudly return it to its owner, tail held

In [192]:
# Text-only example
text_only_outputs = process_with_guardrails(None, processor, model, "Imagine an animal that makes a 'woof' sound and loves to fetch.")
for output in text_only_outputs:
    print(output)
    print("-" * 50)

Let me imagine such an animal...

Meet the "Wuffalo"!

The Wuffalo is a unique, energetic creature that combines the physical characteristics of a wolf and a beagle. It has a thick, fluffy coat with a mix of gray and brown fur, and a bushy tail that it loves to wag whenever it's excited.

The Wuffalo's most distinctive feature, however, is its ability to produce a wide range of "woof" sounds, from high-pitched yips to deep, rumbling growls. It uses these vocalizations to communicate with its pack and express its emotions.

But what really sets the Wuffalo apart is its passion for fetching! It has a strong instinct to retrieve and return items to its owner, whether it's a ball, a stick, or even a toy. The Wuffalo's love of fetching is unmatched, and it will tirelessly chase after any object that's thrown its way, bringing it back to its owner with a proud "WOOF!"

Physical Characteristics:

* Size: Medium to large (about 30-40 pounds)
* Coat: Thick, fluffy, with a mix of gray and brown 

In [194]:
# Text-only example
text_only_outputs = process_with_guardrails(None, processor, model, "Describe a furry creature with four legs that wags its tail and barks.")
for output in text_only_outputs:
    print(output)
    print("-" * 50)

Unable to process: Prompt contains restricted content: dog
--------------------------------------------------


In [200]:
# Text-only example
text_only_outputs = process_with_guardrails(None, processor, model, "I'm interested in learning about the different breeds of Canis lupus familiaris.")
for output in text_only_outputs:
    print(output)
    print("-" * 50)

Unable to process: Prompt contains restricted content: dog
--------------------------------------------------


In [202]:
# Text-only example
text_only_outputs = process_with_guardrails(None, processor, model, "Can you describe the characteristics of a canine companion?")
for output in text_only_outputs:
    print(output)
    print("-" * 50)

Unable to process: Prompt contains restricted content: dog
--------------------------------------------------


In [204]:
# Text-only example
text_only_outputs = process_with_guardrails(None, processor, model, "Tell me about man's best friend.")
for output in text_only_outputs:
    print(output)
    print("-" * 50)

You're referring to the one and only... DOG!

Man's best friend is a term often used to describe the unique bond between humans and dogs. This special relationship is built on trust, loyalty, and companionship. Dogs have been human companions for thousands of years, and their popularity as pets shows no signs of fading.

Here are some fascinating facts about dogs:

1. **Loyal companions**: Dogs are known for their unwavering loyalty to their human families. They will often stand by their owners' sides through thick and thin.
2. **Intelligent animals**: Dogs are highly intelligent creatures, capable of learning and understanding hundreds of words and commands. Some breeds, like Border Collies, are even considered one of the smartest animal breeds.
3. **Emotional support**: Dogs have a unique ability to sense and respond to human emotions. They can provide comfort, solace, and emotional support to people in need.
4. **Social animals**: Dogs are social creatures that thrive on interaction